In [1]:
import pandas as pd
import numpy as np

In [2]:
samples = ['insitu_6_2','insitu_8_1','insitu_9_1','insitu_10_1','insitu_11_1','insitu_12_1',
           'onboard_1_2','onboard_2_2','onboard_7_1','onboard_8_1','onboard_9_1','onboard_10_1','onboard_11_1']

newcol = ['insitu1','insitu2','insitu3','insitu4','insitu5','insitu6', 
          'onboard1','onboard2','onboard3','onboard4','onboard5','onboard6','onboard7']

In [3]:
def Calculate_abundance_and_count():
    
    dfcount=pd.DataFrame()
    dfabundance=pd.DataFrame()
    
    for i in range(len(samples)):
        file = 'Data/' + samples[i] + '_chosen'
        dfsample = pd.read_csv(file, header = 0, usecols=[1,2], sep="\t")
        dfsample.set_index('subject_id', inplace=True)
        
        dfsample['count'] = 1

        count = dfsample.groupby('subject_id').sum() # count
        abundance = dfsample.groupby('subject_id').sum()
        abundance['abundance'] = abundance['count'].apply(lambda x: x / len(dfsample))    # relative abundance

        dfcount = pd.concat([dfcount, count['count']], axis=1, sort=False, join='outer')
        dfabundance = pd.concat([dfabundance, abundance['abundance']], axis=1, sort=False)
            
    dfcount.columns = newcol
    dfabundance.columns = newcol

    return dfcount, dfabundance


In [4]:
dfcount, dfabundance = Calculate_abundance_and_count()
print(len(dfcount))
print(len(dfabundance))

1198
1198


In [5]:
def Add_taxonomy(df):
    
    taxmap = pd.read_csv("/nfs_share/motoki/metatra/slv_DB/taxmap_slv_ssu_123.txt", header=None, sep='\t')
    
    taxmap["subject_id"] = taxmap[0] + '.' + taxmap[1].astype(str) + '.' + taxmap[2].astype(str)
    taxmap = taxmap.set_index(["subject_id"])
    taxmap.columns = ["id", "s_start", "s_end", "taxonomy", "taxonomy2"]
    taxmap.drop(["id","s_start", "s_end"], axis=1, inplace=True)
    
    dftaxonomy = pd.concat([df, taxmap], axis=1, join_axes=[df.index])
    
    return dftaxonomy

In [6]:
dfcount_taxa = Add_taxonomy(dfcount)
dfabundance_taxa = Add_taxonomy(dfabundance)

In [7]:
dfcount_taxa.to_csv('Data/count', sep='\t')
dfabundance_taxa.to_csv('Data/abundance', sep='\t')

In [8]:
dfcount_taxa.head()

,insitu1,insitu2,insitu3,insitu4,insitu5,insitu6,onboard1,onboard2,onboard3,onboard4,onboard5,onboard6,onboard7,taxonomy,taxonomy2
AB239761.1.1412,4.0,3.0,1.0,3.0,4.0,1.0,3.0,8.0,4.0,4.0,12.0,NaN,3.0,Bacteria;Proteobacteria;Gammaproteobacteria;Th...,uncultured bacterium
AB239762.1.1382,2.0,1.0,4.0,4.0,1.0,2.0,NaN,NaN,2.0,1.0,1.0,NaN,2.0,Bacteria;Bacteroidetes;Flavobacteriia;Flavobac...,uncultured bacterium
AB278150.1.1475,1.0,NaN,3.0,2.0,1.0,NaN,1.0,NaN,NaN,NaN,5.0,1.0,1.0,Bacteria;Proteobacteria;Epsilonproteobacteria;...,uncultured bacterium
AB440165.1.1431,29.0,169.0,191.0,190.0,103.0,165.0,28.0,27.0,49.0,41.0,80.0,16.0,27.0,Bacteria;Proteobacteria;Epsilonproteobacteria;...,uncultured bacterium
AB440166.1.1430,1.0,2.0,4.0,2.0,2.0,2.0,2.0,3.0,NaN,4.0,NaN,NaN,1.0,Bacteria;Proteobacteria;Epsilonproteobacteria;...,uncultured bacterium


In [9]:
dfabundance_taxa.head()

,insitu1,insitu2,insitu3,insitu4,insitu5,insitu6,onboard1,onboard2,onboard3,onboard4,onboard5,onboard6,onboard7,taxonomy,taxonomy2
AB239761.1.1412,0.000519,0.000109,0.000039,0.000078,0.000148,0.000039,0.000183,0.000523,0.000105,0.000169,0.000203,NaN,0.000178,Bacteria;Proteobacteria;Gammaproteobacteria;Th...,uncultured bacterium
AB239762.1.1382,0.000259,0.000036,0.000157,0.000105,0.000037,0.000079,NaN,NaN,0.000052,0.000042,0.000017,NaN,0.000118,Bacteria;Bacteroidetes;Flavobacteriia;Flavobac...,uncultured bacterium
AB278150.1.1475,0.000130,NaN,0.000118,0.000052,0.000037,NaN,0.000061,NaN,NaN,NaN,0.000085,0.000062,0.000059,Bacteria;Proteobacteria;Epsilonproteobacteria;...,uncultured bacterium
AB440165.1.1431,0.003759,0.006115,0.007495,0.004965,0.003813,0.006490,0.001710,0.001764,0.001281,0.001735,0.001356,0.000997,0.001598,Bacteria;Proteobacteria;Epsilonproteobacteria;...,uncultured bacterium
AB440166.1.1430,0.000130,0.000072,0.000157,0.000052,0.000074,0.000079,0.000122,0.000196,NaN,0.000169,NaN,NaN,0.000059,Bacteria;Proteobacteria;Epsilonproteobacteria;...,uncultured bacterium
